## Märksõnade (hulga) sobivuse hindamine

Eelnevalt leiti Word2Vec mudelite abil juurde temaatilisi märksõnu. Vaja oleks aga ka hinnata, kas saadud sõnade hulk on piisav. Selleks vaadatakse isikunimega koosesinevaid unikaalseid verbe ning kas ja mil määral esineb leitud märksõnu suvalises unikaalsete verbide alamhulgas.

In [17]:
from estnltk import Text
import sqlite3
from estnltk.converters import json_to_text
import csv
import random

Esmalt tuleb lugeda andmebaasist sisse süntaktilise analüüsiga laused.

In [3]:
sentences = []

In [2]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [4]:
for row in cur.execute("SELECT sentence FROM sentences_syntax_analysis"):
    sentence = json_to_text(json_text=row[0])
    sentences.append(sentence)

In [5]:
con.close()

Järgnevalt leitakse lausetest unikaalsed verbid, mis on seotud isiku- või organisatsiooninimega.

In [11]:
sentence_verbs = set()

In [12]:
for sentence in sentences:
    for entity in sentence.entity_phrases:
        phrase = []
        for span in entity:
            phrase.append(span)
    
        for subspan in phrase:
            parent = sentence.stanza_syntax.get(subspan)['parent_span']
            if parent != None:
                parent_word = sentence['words'].get(parent) 
                if parent_word in phrase:
                    continue
                # Verbist ülemus
                parent_morph = sentence.morph_analysis.get(parent)
                if 'V' not in parent_morph['partofspeech']:
                    continue
                sentence_verbs.add(parent['lemma'])

In [13]:
len(sentence_verbs)

1255

Loetakse sisse temaatilised märksõnad.

In [22]:
thematic_verbs = []

In [23]:
with open('thematic_keywords.csv', encoding='UTF-8') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        if row['lemma'].strip("',") == 'callable':
            continue
        thematic_verbs.append(row['lemma'].strip("',"))

In [25]:
len(thematic_verbs)

96

Järgnevalt võetakse lausetest leitud unikaalsete verbide hulgast juhuslik alamosa, näiteks suurusega 100.

In [26]:
sentence_verbs_list = list(sentence_verbs)

In [27]:
random.shuffle(sentence_verbs_list)

In [28]:
# 100 unikaalse verbi suurune alamosa
sub_100 = sentence_verbs_list[:100]

In [30]:
occurs_in_both = 0

for verb in thematic_verbs:
    if verb in sub_100:
        occurs_in_both += 1

In [32]:
print("Märksõnad moodustasid unikaalsete verbide alamhulgast", (len(sub_100)/100)*occurs_in_both, "protsenti")

Märksõnad moodustasid unikaalsete verbide alamhulgast 5.0 protsenti


Märksõnad moodustasid unikaalsete verbide alamhulgast 5%, mis on suhteliselt ootuspärane arvestades seda, et kogu märksõnade hulk moodustab kõigist unikaalsetest verbidest ~7%.